In [43]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics
from sklearn import ensemble
import sklearn.metrics

In [9]:
!pip install bs4

     |████████████████████████████████| 115 kB 8.0 MB/s eta 0:00:01
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1272 sha256=d31d9d0b18833bf46b9316f0a010a435689457a857b4e175c547200e82b026de
  Stored in directory: /Users/nengneng/Library/Caches/pip/wheels/75/78/21/68b124549c9bdc94f822c02fb9aa3578a669843f9767776bca
Successfully built bs4


In [10]:
# Import necessary libraries
import requests
from bs4 import BeautifulSoup

import datetime

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelBinarizer,OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings("ignore")

In [11]:
# Importing dataset from csv to data frame
train=pd.read_csv('./train.csv')
test=pd.read_csv('./test.csv')
train.head(10)

,id,date,speed
0,0,1/1/2017 0:00,43.002930
1,1,1/1/2017 1:00,46.118696
2,2,1/1/2017 2:00,44.294158
3,3,1/1/2017 3:00,41.067468
4,4,1/1/2017 4:00,46.448653
5,5,1/1/2017 5:00,46.797766
6,6,1/1/2017 6:00,44.404925
7,7,1/1/2017 7:00,45.255897
8,8,1/1/2017 8:00,45.680859
9,9,1/1/2017 9:00,48.435676


In [12]:
trainset=train.copy()
trainset.drop('id',axis=1,inplace=True)

In [13]:
# Split datetime
trainset['year']=trainset['date'].apply(lambda y:int(y.split()[0].split('/')[2]))
trainset['month']=trainset['date'].apply(lambda m:int(m.split()[0].split('/')[1]))
trainset['day']=trainset['date'].apply(lambda d:int(d.split()[0].split('/')[0]))
trainset['time']=trainset['date'].apply(lambda t:int(t.split()[1].split(':')[0]))
trainset['datetime']=trainset['date'].apply(lambda x:
            datetime.datetime(  int(x.split()[0].split('/')[2]),
                int(x.split()[0].split('/')[1]),
                int(x.split()[0].split('/')[0]),
                int(x.split()[1].split(':')[0])  )
                                           )

In [14]:
# Whether a day is weekend or not
trainset['IsWeekend']=trainset['datetime'].apply(lambda x:0 if x.weekday() in [0,1,2,3,4] else 1)

In [15]:
# Whether a day is Hong Kong General Holidays
trainset['IsHoliday']=trainset['datetime'].apply(lambda x:1 if (x.date().strftime('%Y-%m-%d') in [
           '2017-01-02','2017-01-28','2017-01-30','2017-01-31','2017-04-04',
           '2017-04-14','2017-04-15','2017-04-17','2017-05-01','2017-05-03',
           '2017-05-30','2017-07-01','2017-10-02','2017-10-05','2017-10-28',
           '2017-12-25','2017-12-26',
           '2018-01-01','2018-02-16','2018-02-17','2018-02-19','2018-03-30',
           '2018-03-31','2018-04-02','2018-04-05','2018-05-01','2018-05-22',
           '2018-06-18','2018-07-02','2018-09-25','2018-10-01','2018-10-17',
           '2018-12-25','2018-12-26'
           ])
           or(x.weekday() in[6]) else 0)

In [16]:
# Categorizing hours to different time periods like morning, afternoon, etc.
def hour_modify(x):
    Early_Morning = [4,5,6,7]
    Morning = [8,9,10,11]
    Afternoon = [12,13,14,15]
    Evening = [16,17,18,19]
    Night = [20,21,22,23]
    Late_Night = [0,1,2,3]
    if x in Early_Morning:
        return 'Early_Morning'
    elif x in Morning:
        return 'Morning'
    elif x in Afternoon:
        return 'Afternoon'
    elif x in Evening:
        return 'Evening'
    elif x in Night:
        return 'Night'
    else:
        return 'Late_Night'
    
trainset['hour_modify']=trainset['time'].apply(hour_modify)
trainset.head()

,date,speed,year,month,day,time,datetime,IsWeekend,IsHoliday,hour_modify
0,1/1/2017 0:00,43.002930,2017,1,1,0,2017-01-01 00:00:00,1,1,Late_Night
1,1/1/2017 1:00,46.118696,2017,1,1,1,2017-01-01 01:00:00,1,1,Late_Night
2,1/1/2017 2:00,44.294158,2017,1,1,2,2017-01-01 02:00:00,1,1,Late_Night
3,1/1/2017 3:00,41.067468,2017,1,1,3,2017-01-01 03:00:00,1,1,Late_Night
4,1/1/2017 4:00,46.448653,2017,1,1,4,2017-01-01 04:00:00,1,1,Early_Morning


In [22]:
def crawl_weather():
    YEAR=['2017','2018']
    MONTH=['01','02','03','04','05','06','07','08','09','10','11','12']
    URL='https://www.hko.gov.hk/en/wxinfo/pastwx/metob'
    wr=pd.DataFrame({},columns=['year','month','day','MPressure(hPa)',
                  'MaxTemp(℃)','MinTemp(℃)','MCloud(%)',
                  'TRainfall(mm)','#hRedVisi(h)','WindDirect(degrees)',
                  'MWindSpeed(km/h)'],
              index=range(365*2))

    cnt=0
    for yr in YEAR:
        for mon in MONTH:
            url=URL+yr+mon+'.htm'
            ymhtml=requests.get(url)
            soup=BeautifulSoup(ymhtml.text,'lxml')
            tbls=soup.find_all('table')
            tbl0=tbls[0].find_all('tr')
            tbl1=tbls[1].find_all('tr')

            for tr1,tr2 in zip(tbl0[2:-3],tbl1[1:-3]):
                wr.iloc[cnt,0]=yr
                wr.iloc[cnt,1]=mon
                tds1=tr1.find_all('td')
                tds2=tr2.find_all('td')

                wr.iloc[cnt,2]=tds1[0].contents[0]
                wr.iloc[cnt,3]=tds1[1].contents[0]
                wr.iloc[cnt,4]=tds1[2].contents[0]
                wr.iloc[cnt,5]=tds1[4].contents[0]
                wr.iloc[cnt,6]=tds1[7].contents[0]
                wr.iloc[cnt,7]=tds1[8].contents[0]

                wr.iloc[cnt,8]=tds2[1].contents[0]
                wr.iloc[cnt,9]=tds2[5].contents[0]
                wr.iloc[cnt,10]=tds2[6].contents[0]
                cnt+=1
                
    wr.to_csv('./wr.csv')
crawl_weather()

In [23]:
weather=pd.read_csv("./wr.csv")
weather.drop("Unnamed: 0",axis=1,inplace=True)
weather['year'].apply(lambda t:int(t))
weather['month'].apply(lambda t:int(t))
weather['day'].apply(lambda t:int(t))
weather.head()

,year,month,day,MPressure(hPa),MaxTemp(℃),MinTemp(℃),MCloud(%),TRainfall(mm),#hRedVisi(h),WindDirect(degrees),MWindSpeed(km/h)
0,2017,1,1,1021.7,20.8,18.4,72,-,0,60,34.2
1,2017,1,2,1020.2,23.3,18.4,28,-,0,70,17.7
2,2017,1,3,1019.8,21.3,18.9,56,-,5,70,26.1
3,2017,1,4,1018.7,21.7,18.7,51,-,0,70,27.7
4,2017,1,5,1016.9,23.4,18.9,61,-,0,40,14.3


In [24]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   year                 730 non-null    int64  
 1   month                730 non-null    int64  
 2   day                  730 non-null    int64  
 3   MPressure(hPa)       730 non-null    float64
 4   MaxTemp(℃)           730 non-null    float64
 5   MinTemp(℃)           730 non-null    float64
 6   MCloud(%)            730 non-null    int64  
 7   TRainfall(mm)        730 non-null    object 
 8   #hRedVisi(h)         730 non-null    int64  
 9   WindDirect(degrees)  730 non-null    int64  
 10  MWindSpeed(km/h)     730 non-null    float64
dtypes: float64(4), int64(6), object(1)
memory usage: 62.9+ KB


In [25]:
weather.drop(['year','month','day'],axis=1).describe()

,MPressure(hPa),MaxTemp(℃),MinTemp(℃),MCloud(%),#hRedVisi(h),WindDirect(degrees),MWindSpeed(km/h)
count,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000
mean,1012.883562,26.564110,22.012877,70.315068,1.032877,140.671233,23.796849
std,6.504514,5.354513,5.088792,22.101572,2.924569,110.679229,10.542179
min,990.900000,10.600000,6.800000,4.000000,0.000000,10.000000,4.000000
25%,1008.400000,22.700000,17.900000,59.000000,0.000000,60.000000,15.900000
50%,1013.400000,27.400000,23.000000,79.000000,0.000000,80.000000,23.600000
75%,1017.500000,31.100000,26.400000,88.000000,0.000000,220.000000,30.650000
max,1028.200000,36.600000,29.800000,100.000000,21.000000,360.000000,101.300000


In [26]:
weather['TRainfall(mm)'].value_counts()

-        297
Trace    153
0.2       17
0.1       15
0.3       13
        ... 
7.8        1
28.2       1
24.9       1
33.4       1
26.4       1
Name: TRainfall(mm), Length: 174, dtype: int64

In [27]:
# Clean the Na 
weather['TRainfall(mm)_num']=weather['TRainfall(mm)'].apply(lambda x:0 if x in ['-','Trace'] else x)


In [28]:
trainset['common']=trainset['datetime'].apply(lambda x:x.date())
def addup(x):
    return datetime.datetime(x[0],x[1],x[2]).date()
weather['common']=weather.apply(addup,axis=1)

data=pd.merge(trainset[['datetime','year','month','day','time','speed','IsWeekend','IsHoliday','hour_modify','common']],
              weather[['MaxTemp(℃)','MinTemp(℃)','MCloud(%)','TRainfall(mm)_num','#hRedVisi(h)','WindDirect(degrees)','MWindSpeed(km/h)','common']],on='common',how='left')\

data.head()

,datetime,year,month,day,time,speed,IsWeekend,IsHoliday,hour_modify,common,MaxTemp(℃),MinTemp(℃),MCloud(%),TRainfall(mm)_num,#hRedVisi(h),WindDirect(degrees),MWindSpeed(km/h)
0,2017-01-01 00:00:00,2017,1,1,0,43.002930,1,1,Late_Night,2017-01-01,20.8,18.4,72,0,0,60,34.2
1,2017-01-01 01:00:00,2017,1,1,1,46.118696,1,1,Late_Night,2017-01-01,20.8,18.4,72,0,0,60,34.2
2,2017-01-01 02:00:00,2017,1,1,2,44.294158,1,1,Late_Night,2017-01-01,20.8,18.4,72,0,0,60,34.2
3,2017-01-01 03:00:00,2017,1,1,3,41.067468,1,1,Late_Night,2017-01-01,20.8,18.4,72,0,0,60,34.2
4,2017-01-01 04:00:00,2017,1,1,4,46.448653,1,1,Early_Morning,2017-01-01,20.8,18.4,72,0,0,60,34.2


In [30]:
data_taken_for_train=data.drop(['datetime','common'],axis=1)
data_taken_for_train.head()

,year,month,day,time,speed,IsWeekend,IsHoliday,hour_modify,MaxTemp(℃),MinTemp(℃),MCloud(%),TRainfall(mm)_num,#hRedVisi(h),WindDirect(degrees),MWindSpeed(km/h)
0,2017,1,1,0,43.002930,1,1,Late_Night,20.8,18.4,72,0,0,60,34.2
1,2017,1,1,1,46.118696,1,1,Late_Night,20.8,18.4,72,0,0,60,34.2
2,2017,1,1,2,44.294158,1,1,Late_Night,20.8,18.4,72,0,0,60,34.2
3,2017,1,1,3,41.067468,1,1,Late_Night,20.8,18.4,72,0,0,60,34.2
4,2017,1,1,4,46.448653,1,1,Early_Morning,20.8,18.4,72,0,0,60,34.2


In [31]:
data_taken_for_train.columns

Index(['year', 'month', 'day', 'time', 'speed', 'IsWeekend', 'IsHoliday',
       'hour_modify', 'MaxTemp(℃)', 'MinTemp(℃)', 'MCloud(%)',
       'TRainfall(mm)_num', '#hRedVisi(h)', 'WindDirect(degrees)',
       'MWindSpeed(km/h)'],
      dtype='object')

In [103]:
data_taken_for_train=data_taken_for_train.drop(['MaxTemp(℃)','MinTemp(℃)','MCloud(%)','TRainfall(mm)_num','#hRedVisi(h)','WindDirect(degrees)','MWindSpeed(km/h)'],axis=1)

In [104]:
data_taken_for_train.head()

,year,month,day,time,speed,IsWeekend,IsHoliday,hour_modify
0,2017,1,1,0,43.002930,1,1,Late_Night
1,2017,1,1,1,46.118696,1,1,Late_Night
2,2017,1,1,2,44.294158,1,1,Late_Night
3,2017,1,1,3,41.067468,1,1,Late_Night
4,2017,1,1,4,46.448653,1,1,Early_Morning


In [107]:
target=['speed']
cat_vars=['IsWeekend','IsHoliday','hour_modify']
# num_vars=['year','month','day','time','MaxTemp(℃)','MinTemp(℃)','MCloud(%)',
#           'TRainfall(mm)_num','#hRedVisi(h)','WindDirect(degrees)',
#           'MWindSpeed(km/h)']
num_vars=['year','month','day','time']

In [108]:
# Creating pipeline to transform data
numeric_transformer=Pipeline(steps=[
    ('scaler', StandardScaler())])
categorical_transformer=Pipeline(steps=[
    ('oneHot',OneHotEncoder())])

preprocessor=ColumnTransformer(transformers=[
    ('num',numeric_transformer,num_vars),
    ('cat',categorical_transformer,cat_vars)])

data_transformed=preprocessor.fit_transform(data_taken_for_train)

In [109]:
# Split the data into trainset and testset.

# direction 1
X_train,X_test,y_train,y_test=train_test_split(data_transformed,data_taken_for_train['speed'],test_size=0.15,random_state=42)

In [110]:
print('Shape of X_train: ',str(X_train.shape));print('Shape of y_train: ',str(y_train.shape))
print('Shape of X_test: ',str(X_test.shape));print('Shape of y_test: ',str(y_test.shape))

Shape of X_train:  (11905, 14)
Shape of y_train:  (11905,)
Shape of X_test:  (2101, 14)
Shape of y_test:  (2101,)


In [111]:

model = XGBRegressor()

# param_grid = {'nthread':[4], 
#               'objective':['reg:squarederror'],
#               'learning_rate':[0.01],
#               'max_depth':[8,12,15],
#               'min_child_weight':[4,6],
#               'subsample':[0.9],
#               'colsample_bytree':[0.7],
#               'n_estimators':[500,1000],
#               'reg_lambda':[0.1],
#               'gamma':[0.1,0]}

param_grid = {'learning_rate': [0.07],'reg_alpha': [3], 
              'reg_lambda': [3],'subsample': [1], 
              'colsample_bytree': [0.8],'gamma': [0.2],
              'max_depth': [7], 'min_child_weight': [4],'n_estimators':[650], 'n_estimators': [500,1000],  'seed': [0],}
                   

GridSearch=GridSearchCV(estimator=model,param_grid=param_grid,cv=5,n_jobs=4)
GridSearch.fit(X_train,y_train)
y_pred=GridSearch.predict(X_test)
print(GridSearch.best_estimator_)


# from sklearn.model_selection import GridSearchCV 

# if __name__ == '__main__':

#     cv_params = {'learning_rate': 0.07}
#     other_params = {'learning_rate': 0.07,'reg_alpha': 3, 'reg_lambda': 3,'subsample': 1, 'colsample_bytree': 0.8,'gamma': 0.2,'max_depth': 7, 'min_child_weight': 4,'n_estimators':650, 'n_estimators': 500,  'seed': 0,
#                     }

#     model = xgb.XGBRegressor(**other_params)
#     optimized_GBM = GridSearchCV(estimator=model, param_grid=cv_params, scoring='r2', cv=5, verbose=1, n_jobs=4)
#     optimized_GBM.fit(X_train, y_train)
#     evalute_result = optimized_GBM.cv_results_['mean_test_score']
#     print('每轮迭代运行结果:{0}'.format(evalute_result))
#     print('参数的最佳取值：{0}'.format(optimized_GBM.cv_results_['params']))
#     print('最佳模型得分:{0}'.format(optimized_GBM.cv_results_['mean_test_score']))



XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, gamma=0.2, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.07, max_delta_step=0, max_depth=7,
             min_child_weight=4, missing=nan, monotone_constraints='()',
             n_estimators=500, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=3, reg_lambda=3, scale_pos_weight=1, seed=0, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)


In [112]:
MSE_xgb=mean_squared_error(y_pred,y_test)  # 10.7063378519613

print('-'*15,'Direction 1','-'*15)
print('MSE for XGBoost is '+str(MSE_xgb))


--------------- Direction 1 ---------------
MSE for XGBoost is 11.007684793516734


In [101]:
# Prepare for prediction
testset=test.copy()
testset.drop('id',axis=1,inplace=True)

testset['year']=testset['date'].apply(lambda y:int(y.split()[0].split('/')[2]))
testset['month']=testset['date'].apply(lambda m:int(m.split()[0].split('/')[1]))
testset['day']=testset['date'].apply(lambda d:int(d.split()[0].split('/')[0]))
testset['time']=testset['date'].apply(lambda t:int(t.split()[1].split(':')[0]))
testset['datetime']=testset['date'].apply(lambda x:
           datetime.datetime(  int(x.split()[0].split('/')[2]),
           int(x.split()[0].split('/')[1]),
           int(x.split()[0].split('/')[0]),
           int(x.split()[1].split(':')[0])  )
           )
testset['IsWeekend']=testset['datetime'].apply(lambda x:0 if x.weekday() in [0,1,2,3,4] else 1)
testset['IsHoliday']=testset['datetime'].apply(lambda x:1 if (x.date().strftime('%Y-%m-%d') in [
           '2017-01-02','2017-01-28','2017-01-30','2017-01-31','2017-04-04',
           '2017-04-14','2017-04-15','2017-04-17','2017-05-01','2017-05-03',
           '2017-05-30','2017-07-01','2017-10-02','2017-10-05','2017-10-28',
           '2017-12-25','2017-12-26',
           '2018-01-01','2018-02-16','2018-02-17','2018-02-19','2018-03-30',
           '2018-03-31','2018-04-02','2018-04-05','2018-05-01','2018-05-22',
           '2018-06-18','2018-07-02','2018-09-25','2018-10-01','2018-10-17',
           '2018-12-25','2018-12-26'
           ])
           or(x.weekday() in[6]) else 0)

testset['hour_modify']=testset['time'].apply(hour_modify)

testset['common']=testset['datetime'].apply(lambda x:x.date())

testdata=pd.merge(testset[['datetime','year','month','day','time','IsWeekend','IsHoliday','hour_modify','common']],
              weather[['MaxTemp(℃)','MinTemp(℃)','MCloud(%)','TRainfall(mm)_num','#hRedVisi(h)','WindDirect(degrees)','MWindSpeed(km/h)','common']],on='common',how='left')\

testdata.head()

# Prediction
test_transformed=preprocessor.fit_transform(testdata)
y_test_pred=GridSearch.predict(test_transformed)

print(len(y_test_pred))
test_transformed.shape[0]

# Write into test.casv and save as submission.csv

pred=pd.DataFrame({'speed':list(y_test_pred)},columns=['speed'])
submission=pd.concat([test.drop('date',axis=1),pred],axis=1)
submission.head()

3504


,id,speed
0,0,47.736820
1,1,46.816406
2,2,38.846893
3,3,31.638952
4,4,40.432972


In [102]:
submission.to_csv('submission17.csv',index=False)